In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
fn = 'Batch-2-HiChIP-Samples-8.16.22-10.23.23-Samplesheet.tsv'

In [3]:
df = pd.read_table(fn)

In [4]:
df.loc[:, 'original_location'] = np.arange(df.shape[0])

In [5]:
df

,gse_id,gse_id_new,gsm_id,srr_id,geo_title,geo_source,geo_description,organism,num_reads,study,...,tech_rep_new,Unnamed: 14,bio_eq,tech_eq,chip_pulldown,restriction_enzyme,shade,sample_name,Double Check Later,original_location
0,GSE123645,NaN,GSM3508600,SRR8312202,H3K27ac HiChIP C666-1 Replicate 1,EBV-associated nasopharyngeal carcinoma cell line,description,Homo sapiens,24715122.0,NaN,...,1.0,NaN,1,1,H3K27ac,MboI,1.0,C666-1,NaN,0
1,GSE123645,NaN,GSM3508601,SRR8312203,H3K27ac HiChIP C666-1 Replicate 2,EBV-associated nasopharyngeal carcinoma cell line,description,Homo sapiens,47922136.0,NaN,...,1.0,NaN,1,1,H3K27ac,MboI,1.0,C666-1,NaN,1
2,GSE123645,NaN,GSM3508602,SRR8312204,H3K27ac HiChIP C17 Replicate 1,EBV-associated nasopharyngeal carcinoma cell line,description,Homo sapiens,40703228.0,NaN,...,1.0,NaN,1,1,H3K27ac,MboI,1.0,C17,NaN,2
3,GSE123645,NaN,GSM3508603,SRR8312205,H3K27ac HiChIP C17 Replicate 2,EBV-associated nasopharyngeal carcinoma cell line,description,Homo sapiens,56469726.0,NaN,...,1.0,NaN,1,1,H3K27ac,MboI,1.0,C17,NaN,3
4,GSE128952,NaN,GSM3688946,SRR8795606,EBNA3A-HT HiChIP H3K27ac 3A On Replicate 1,EBNA3A-HT Lymphoblastoid Cell Lines,description,Homo sapiens,34742638.0,"Wang et al., 2023",...,1.0,NaN,1,1,H3K27ac,MboI,0.0,LCL-EBNA3A-HT-On,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,GSE243915,GSE243913,GSM7798272,SRR26158121,"LOUCY-DMSO, H3K27ac (HiChIP)",LOUCY,NaN,Homo sapiens,174179758.0,"Javier et al., 2023",...,3.0,NaN,1,1,H3K27ac,Arima,0.0,LOUCY-DMSO,NaN,480
481,GSE243915,GSE243913,GSM7798272,SRR26158122,"LOUCY-DMSO, H3K27ac (HiChIP)",LOUCY,NaN,Homo sapiens,174179758.0,"Javier et al., 2023",...,4.0,NaN,1,1,H3K27ac,Arima,0.0,LOUCY-DMSO,NaN,481
482,GSE245495,NaN,GSM7843893,SRR26400235,"RD, H3K27ac, HiChIP",RD,NaN,Homo sapiens,830100728.0,Pomella et al. Biorxiv,...,1.0,NaN,1,1,H3K27ac,MNase,1.0,RD,NaN,482
483,GSE245495,NaN,GSM7843894,SRR26400234,"RH4, H3K27ac, HiChIP",RH4,NaN,Homo sapiens,716828368.0,Pomella et al. Biorxiv,...,1.0,NaN,1,1,H3K27ac,MNase,1.0,RH4,NaN,483


In [6]:
sorted_df = df.sort_values(['gse_id', 'gsm_id', 'srr_id'])

In [15]:
bio_reps_all = []
tech_reps_all = []
original_idxs = []

expected_count = 0

# splitting by gse
for gse_grp, gse_df in sorted_df.groupby('gse_id'):
    
    expected_count += gse_df.shape[0]
    
    #print('gse_df: {}'.format(len(gse_df)))
    
    # splitting by sample_name (same biological sample)
    for (sn_grp, pd_grp), sn_df in gse_df.groupby(['sample_name', 'chip_pulldown']):
        
        
        #print('\tsn_df: {}'.format(len(sn_df)))

        
        # splitting by gse_id, each one represents a biological replicate
        bio_rep = 1
        bio_reps = []
        tech_reps = []
        for gsm_grp, gsm_df in sn_df.groupby('gsm_id'):
            
                
            #print('\t\tgsm_df: {}'.format(len(gsm_df)))
    
            
            # adding the biological replicates
            bio_reps.extend(['{}'.format(bio_rep) for x in range(gsm_df.shape[0])])
            
            # creating distinct technical replicates for each SRR
            tech_reps.extend(['{}'.format(x + 1) for x in range(gsm_df.shape[0])])
            
            original_idxs.extend(gsm_df.original_location.tolist())
        
            # updating the bio_rep
            bio_rep += 1 
        
        # recording replicates for all samples
        bio_reps_all.extend(bio_reps)
        tech_reps_all.extend(tech_reps)
        
    if len(bio_reps_all) != expected_count:
        print(len(bio_reps_all))
        print(expected_count)
        print('broken')
        print('\n\n')
        break
        

In [16]:
reps_df = pd.DataFrame([bio_reps_all, tech_reps_all, original_idxs]).T

In [17]:
reps_df.columns = ['bio_rep', 'tech_rep', 'original_location']

In [18]:
reps_df

,bio_rep,tech_rep,original_location
0,1,1,2
1,2,1,3
2,1,1,0
3,2,1,1
4,1,1,8
...,...,...,...
479,1,1,462
480,1,1,467
481,1,1,468
482,1,1,482


In [19]:
reps_df

,bio_rep,tech_rep,original_location
0,1,1,2
1,2,1,3
2,1,1,0
3,2,1,1
4,1,1,8
...,...,...,...
479,1,1,462
480,1,1,467
481,1,1,468
482,1,1,482


In [20]:
new_df = df.merge(reps_df, on='original_location', suffixes=['_old', '_new'], how='outer', indicator=True)

In [21]:
new_df.loc[new_df._merge != 'both']

,gse_id,gse_id_new,gsm_id,srr_id,geo_title,geo_source,geo_description,organism,num_reads,study,...,tech_eq,chip_pulldown,restriction_enzyme,shade,sample_name,Double Check Later,original_location,bio_rep_new,tech_rep_new,_merge
484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,389,NaN,NaN,NaN,NaN,NaN,484,NaN,NaN,left_only


In [22]:
new_df.to_csv('test.tsv', sep='\t', index=False, header=True)